<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Terraclimate downscaling

_by Oriana Chegwidden (CarbonPlan), December 15, 2020_

This notebook evaluates the downscaling of CMIP6 variables against the obs
Terraclimate dataset.


In [ ]:
%load_ext autoreload
%autoreload 2

from cmip6_downscaling.analysis import load, metrics, plot
import dask
import os
import matplotlib.pyplot as plt

In [ ]:
from dask_gateway import Gateway

gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores = 2
options.worker_memory = 24
cluster = gateway.new_cluster(cluster_options=options)
cluster.adapt(minimum=10, maximum=20)
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
# load obs data
obs = load.load_obs()
# load cmip data
gcm = load.load_cmip(model="CanESM5", scenario="historical", member="r10i1p1f1")
variables_to_plot = ["ppt", "pdsi", "pet", "rh", "vpd", "tmax", "tmin"]
# calculate a metric from existing ones
# make a plot comparing cmip and obs

Let's brainstorm a bunch of things we'll want to plot.


In [ ]:
time_slice = slice("1970", "1999")

In [ ]:
results_obs = metrics.calc(obs[variables_to_plot].sel(time=time_slice), compute=True)

In [ ]:
results_gcm = metrics.calc(gcm[variables_to_plot].sel(time=time_slice), compute=True)

The goal of these analyses is to evaluate the effect of different steps in the
modeling chain. By teasing out where bias is introduced or where the
distribution gets distorted, we'll better understand our output met datasets.


Future climate plotting - within each of the historical timeseries, what
influence does a future climate have on each of the variables. Spatial plots


Make a multivariable map plot with columns being (left) obs (center) gcm and
(right) the difference. Rows are the different variabless selected via
`variables_to_plot`.

Plotting requirements:

- averaging time period
- gcm/ensemble/historical
- standardize colorbars for the obs/historical


Some of the variables throw wierd numbers so for plotting we'll constrain them
to a reasonable limit.


In [ ]:
limits_dict = {"pdsi": {"vmin": -16, "vmax": 16}}

Seems that the analyses like it when there are 6 workers minimum, so let's check
out at least that many once we start to run all of our analyses so that we can
be speedy from the get-go.


In [ ]:
fig, axarr = plot_time_mean(
    results_obs["time_mean"],
    results_gcm["time_mean"],
    diff=True,
    limits_dict=limits_dict,
)

Now let's also do some seasonal mean plotting. Maybe the hub will behave better
for this one.


Spatial autocorrelation analyses Things to think about:

- what weighting scheme
- how many spatiotemporal lags to have


In [ ]:
import pysal
from esda.moran import Moran

mi = Moran(y, w)